# 시계열 데이터란?    
* 시계열 : 일정시간 간격으로 배치된 데이터들의 수열
* 시계열 데이터 : 시계열의 규칙을 갖고있는 데이터

![ex_screenshot](./image/time-series.png)


## 시계열 데이터 분석 방법
1. ARIMA 
2. RNN (Reccurent Neural Networks)
3. LSTM (Long Short-Term Memory models)

## RNN이란?    
이전 입력에 대한 정보를 현재의 입력에 대한 분석에 활용하는 은닉층이 하나인 Neural Network.    
#### 장점
1. 순환 데이터나, 시간성(Temporal Property)등 시간성 정보를 잘 이용해야 하는 데이터 분석에 강하다.
2. 일정하지 않은 input에 대해 분석이 가능하다.     

####  단점
1. 데이터의 time - series가 길어짐에따라 계산하는데 걸리는 시간이 증가한다.    
    -> RNN의 간소화된 버전 GRU(Gated Recurrent Unit)를 통해 해결
2. Vanishing Gradient Problem    
    -> RNN과 같이 단순한 neural network 한 층 대신, 정보를 전달하는 4개의 layer를 갖는 LSTM으로 해결    
    
## Vanishing Gradient Problem (장기 의존성)이란?    
Input데이터의 길이가 길어짐으로인해, time step도 마찬가지로 길어지면서, 비교적 앞부분에 대한 Hidden state정보를 전달하지 못하는 문제

![title = RNN과 LSTM의 구조적 차이](./image/RNNnLSTM.png)

## LSTM이란?    
RNN의 일종으로, RNN의 장기 의존성(Long-Term Dependency)문제를 해결하기위해 제안된 방안

![title = RNN과 LSTM의 구조적 차이](./image/LSTM.png)

forget gate: 이전 hidden state와 t번째 input을 고려해서 어떤것을 잊어버릴지.    
input gate: 이전 hidden state와 t번째 input을 고려해서 어떤것을 기억할지.    

(핵심)Cell state update: forget gate의 값과 input gate의 값을 조합해서 최종적으로 cell state를 업데이트    
output gate: hidden state와 t번째 input을 고려해서 다음 hidden state를 계산

데이터 분석을 통해 각 세대의 사용량을 예측하여 잘못된 검침값에 대한 선제적 조치를 취하자.    
train.csv : 2009년 1월 - 2019년 5월 / 78587가구    
test.csv : 2009년 1월 - 2018년 5월 / 10000가구    
submission.csv : 2018년 6월 - 2019년 5월 / 10000가구    
분석 방법 : LSTM

### 모듈 및 데이터 불러오기

In [9]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt

### 데이터 확인 및 정제

In [17]:
df_1 = pd.read_csv('train.csv')
df_2 = pd.read_csv('test.csv')
df_3 = pd.read_csv('sample_submission.csv')
print('train shape = ',df_1.shape)
print('test shape = ',df_2.shape)
print('submission shape = ',df_3.shape)

train shape =  (78587, 126)
test shape =  (10000, 114)
submission shape =  (10000, 13)


In [20]:
df_1.columns

Index(['id', '2009-01', '2009-02', '2009-03', '2009-04', '2009-05', '2009-06',
       '2009-07', '2009-08', '2009-09',
       ...
       '2018-08', '2018-09', '2018-10', '2018-11', '2018-12', '2019-01',
       '2019-02', '2019-03', '2019-04', '2019-05'],
      dtype='object', length=126)

In [57]:
train_x = df_1.loc[:,'2009-01':'2018-05']
train_y = df_1.loc[:,'2018-06':'2019-05']
print('train_x.shape : ',train_x.shape)
print('train_y.shape : ',train_y.shape)

train_x.shape :  (78587, 113)
train_y.shape :  (78587, 12)


In [58]:
X_train = np.array(train_x).reshape(train_x.shape[0], train_x.shape[1],1)
y_train = np.array(train_y).reshape(train_y.shape[0], train_y.shape[1])
print('X_train.shape : ',X_train.shape)
print('y_train.shape : ',y_train.shape)

X_train.shape :  (78587, 113, 1)
y_train.shape :  (78587, 12)


![title = RNN과 LSTM의 구조적 차이](./image/shape.png)

### lstm 학습 모듈 불러오기

In [59]:
from keras.layers import LSTM
from keras.models import Sequential
from keras.layers import Dense
import keras.backend as K
from keras.callbacks import EarlyStopping

### 모델 만들기

In [65]:
model = Sequential()
model.add(LSTM(20, input_shape = (113,1))) #(timestep, feature)
model.add(Dense(12)) # output = 1
model.compile(loss = 'mean_squared_error', optimizer ='adam')

model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_7 (LSTM)                (None, 20)                1760      
_________________________________________________________________
dense_7 (Dense)              (None, 12)                252       
Total params: 2,012
Trainable params: 2,012
Non-trainable params: 0
_________________________________________________________________


### 모델 학습

In [ ]:
early_stop = EarlyStopping(monitor = 'loss', patience = 1, verbose = 1)

hist = model.fit(X_train, y_train, epochs = 100, 
         batch_size = 32, verbose = 1, callbacks = [early_stop])

Epoch 1/100
78587/78587 [==============================] - 126s 2ms/step - loss: 3636.3607
Epoch 2/100
78587/78587 [==============================] - 125s 2ms/step - loss: 2130.5152
Epoch 3/100
78587/78587 [==============================] - 126s 2ms/step - loss: 1543.9169
Epoch 4/100
78587/78587 [==============================] - 140s 2ms/step - loss: 1288.4232
Epoch 5/100
78587/78587 [==============================] - 141s 2ms/step - loss: 1178.9970
Epoch 6/100
78587/78587 [==============================] - 141s 2ms/step - loss: 1128.4589
Epoch 7/100
78587/78587 [==============================] - 140s 2ms/step - loss: 1099.0105
Epoch 8/100
78587/78587 [==============================] - 141s 2ms/step - loss: 1079.4818
Epoch 9/100
78587/78587 [==============================] - 141s 2ms/step - loss: 1065.0157
Epoch 10/100
78587/78587 [==============================] - 141s 2ms/step - loss: 1054.2982
Epoch 11/100
78587/78587 [==============================] - 140s 2ms/step - loss: 1048.02

### 학습 결과 확인